In [1]:
%load_ext autoreload
%autoreload 2

## Pytorch EO Tasks

Pytorch EO core design revolves around `Tasks`. Examples of tasks are: image classification, object detection, image segmentation, etc.

A task is defined by:
1. `model`: a neural network that will be trained to perform the task
2. `hparams`: hyperparameters used to train the model (optimizer, scheduler and additional information)
3. `loss function`: criterion used during the optimization process
4. `metrics`: evaluation of the model in the task
5. `inputs` and `outputs`: lists with the names of the inputs-outputs

## Models

Models are neural networks that take inputs and produce outputs. You can build your own or use third party models. By default we use `torchvision`, so you can use any model from there

In [2]:
import torch 
from pytorch_eo.tasks.BaseTask import BaseTask

task = BaseTask('resnet18')

output = task(torch.randn(32,3,224,224))
output.shape

/home/juan/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


torch.Size([32, 1000])

In case you use models from torchvision, you can pass any parameter in the `hparams` object as follows.

In [3]:
hparams = {
    'model': {
        'weights': 'ResNet18_Weights.IMAGENET1K_V1'
    }
}

task = BaseTask('resnet18', hparams)

output = task(torch.randn(32,3,224,224))
output.shape

torch.Size([32, 1000])

You can always make your own models, for example modifying an existing model from torchvision.

In [4]:
import torch
import torchvision 

model = torchvision.models.resnet18()
model.fc = torch.nn.Linear(512, 10)

task = BaseTask(model)

output = task(torch.randn(32,3,224,224))
output.shape

torch.Size([32, 10])

Or building your own model from scratch.

In [5]:
class MyModel(torch.nn.Module):
    def __init__(self, pretrained, num_classes):
        super().__init__()
        resnet = torchvision.models.resnet18(pretrained=pretrained)
        self.backbone = torch.nn.Sequential(*list(resnet.children())[:-2])
        self.fc = torch.nn.Sequential(
            torch.nn.AdaptiveAvgPool2d((1, 1)),
            torch.nn.Flatten(),
            torch.nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        f = self.backbone(x)
        return self.fc(f)

In [6]:
model = MyModel(pretrained=True, num_classes=10)

task = BaseTask(model)

output = task(torch.randn(32, 3, 224,224))
output.shape

/home/juan/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/juan/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([32, 10])

But you would probably want to use third party implementations with extra functionality.

In [7]:
import timm 

model = timm.create_model(
    'resnet18',
    pretrained='imagenet',
    in_chans=3,
    num_classes=10,
    features_only=True
)

task = BaseTask(model)

output = task(torch.randn(32,3,224,224))
for o in output:
    print(o.shape)

torch.Size([32, 64, 112, 112])
torch.Size([32, 64, 56, 56])
torch.Size([32, 128, 28, 28])
torch.Size([32, 256, 14, 14])
torch.Size([32, 512, 7, 7])


In [8]:
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name='resnet18',
    encoder_weights='imagenet',
    in_channels=3,
    classes=2,
)

task = BaseTask(model)

output = task(torch.randn(32,3,224,224))
output.shape

torch.Size([32, 2, 224, 224])

## hparams

This is a `dict` holding the hyperparameters used for training. Pytorch lightning will save the object in the model's checkpoint (that can be used to resume training, for example) so make sure to add any additional information that you want to save. In some cases, default parameters will be used if hparams are not provided.

In [9]:
hparams = {
    # hparams used for training
    'model': { # only if you use default torchvision models
        'weights': 'ResNet18_Weights.IMAGENET1K_V1'
    },
    'loss': 'CrossEntropyLoss', # choose one from pytorch docs
    'loss_params': {}, 
    'optimizer': 'Adam', # choose one from pytorch docs
    'optim_params': {
        'lr': 1e-4
    },
    'scheduler': 'CosineAnnealingLR', # choose one from pytorch docs
    'scheduler_params': {
        'T_max': 10,
        'verbose': True
    }
    # extra
    # epochs, batch size, model, transforms, ...
}

## Loss Function

You can specify the loss function to use during training as follows

In [10]:
import pytorch_lightning as pl
from pytorch_eo.datasets import EuroSATRGB
from pytorch_eo.tasks import ImageClassification
from pytorch_eo.metrics.classification import accuracy
import albumentations as A
from albumentations.pytorch import ToTensorV2

trans = A.Compose([
    A.Normalize(0, 1),
    ToTensorV2(),
])

ds = EuroSATRGB(batch_size=32, train_trans=trans, val_trans=trans)

model = torchvision.models.resnet18()
model.fc = torch.nn.Linear(512, 10)

task = ImageClassification(model, loss_fn=torch.nn.CrossEntropyLoss())

trainer = pl.Trainer(
    gpus=1,
    precision=16,
    max_epochs=3,
    limit_train_batches=10
)

trainer.fit(task, ds)

/home/juan/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:467: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/juan/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_conn

Sanity Checking: 0it [00:00, ?it/s]

/home/juan/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/juan/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/juan/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1600: PossibleUserWarning: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_st

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


## Metrics

The last piece to train a model is the metrics. You can train without metrics, use our simple convenient implementations, or use your own implementations. In most tasks we will use at least one metric even if they are not provided (they are that important !)

In [11]:
import pytorch_lightning as pl

from pytorch_eo.datasets import EuroSATRGB
from pytorch_eo.tasks import ImageClassification
from pytorch_eo.metrics.classification import accuracy

ds = EuroSATRGB(batch_size=32, train_trans=trans, val_trans=trans)

model = torchvision.models.resnet18()
model.fc = torch.nn.Linear(512, 10)

metrics = {'acc': accuracy}

task = ImageClassification(model, metrics=metrics)

In [12]:
trainer = pl.Trainer(
    gpus=1,
    precision=16,
    max_epochs=3,
    limit_train_batches=10
)

trainer.fit(task, ds)

/home/juan/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:467: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type             | Params
---------------------------------------------

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


You can use as many metrics as you want (all will be logged)

In [13]:
def my_accuracy(y_hat, y):
    return (torch.argmax(y_hat, axis=1) == y).sum() / y.shape[0]

metrics = {'acc': accuracy, 'my_acc': my_accuracy}

task = ImageClassification(model, hparams, metrics=metrics)

In [14]:
trainer = pl.Trainer(
    gpus=1,
    precision=16,
    max_epochs=3,
    limit_train_batches=10
)

trainer.fit(task, ds)

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type             | Params
---------------------------------------------
0 | model   | ResNet           | 11.2 M
1 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
22.363    Total estimated model params size (MB)


Adjusting learning rate of group 0 to 1.0000e-04.


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Adjusting learning rate of group 0 to 9.7553e-05.


...

Adjusting learning rate of group 0 to 9.0451e-05.


Validation: 0it [00:00, ?it/s]

Adjusting learning rate of group 0 to 7.9389e-05.


Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


But you would probably want to use a third party library like `torchmetrics` with a lot of tested implementations.

In [19]:
from torchmetrics.functional import accuracy as tm_accuracy

# torchmetrics wants probas
def my_tm_accuracy(y_pred, y):
    y_pred = torch.softmax(y_pred, 1)
    return tm_accuracy(y_pred, y, task="multiclass", num_classes=10)

metrics = {'acc': accuracy, 'my_acc': my_accuracy, 'tm_acc': my_tm_accuracy}

task = ImageClassification(model, hparams, metrics=metrics)

In [20]:
trainer = pl.Trainer(
    gpus=1,
    precision=16,
    max_epochs=3,
    limit_train_batches=10
)

trainer.fit(task, ds)

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type             | Params
---------------------------------------------
0 | model   | ResNet           | 11.2 M
1 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
22.363    Total estimated model params size (MB)


Adjusting learning rate of group 0 to 1.0000e-04.


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Adjusting learning rate of group 0 to 9.7553e-05.


...

Adjusting learning rate of group 0 to 9.0451e-05.


Validation: 0it [00:00, ?it/s]

Adjusting learning rate of group 0 to 7.9389e-05.


Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


## Inputs and Outputs

PytorchEO datasets are designed to be flexible in the number and modality of inputs and outputs in order to allow for advanced applications such as data fusion, unsupervised learning or multi-task learning. For this reason, you must tell the task which fields in your datasets have to be used for inputs and outputs, respectively.

In [ ]:
import os
from pytorch_eo.utils import download_url, unzip_file
from pathlib import Path
from pytorch_eo.datasets import RGBImageDataset
from pytorch_eo.datasets import ConcatDataset
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from torch.utils.data import DataLoader

def prepare_data(path):
	os.makedirs(path, exist_ok=True)
	url = "http://madm.dfki.de/files/sentinel/EuroSAT.zip"
	compressed_data_filename = 'EuroSAT.zip'
	data_folder = '2750'
	compressed_data_path = path / compressed_data_filename
	download_url(url, compressed_data_path)
	unzip_file(compressed_data_path, path, msg="extracting data ...")
	uncompressed_data_path = path / data_folder
	classes = sorted(os.listdir(uncompressed_data_path))
	images, labels = [], []
	for ix, label in enumerate(classes):
		_images = os.listdir(uncompressed_data_path / label)
		images += [str(uncompressed_data_path /
					label / img) for img in _images]
		labels += [ix]*len(_images)
	return images, labels

class EuroSATDataset(pl.LightningDataModule):
	def __init__(self, path='data', trans=None):
		super().__init__()
		self.path = Path(path)
		self.trans = trans

	def setup(self, stage=None):
		images, labels = prepare_data(self.path)
		images_ds = RGBImageDataset(images) 
		# Custom keys in the dataset !
		self.ds = ConcatDataset({'abc': images_ds, 'def': labels}, trans=self.trans, image_key='abc')

	def train_dataloader(self):
		return DataLoader(self.ds)

In [ ]:
trans = A.Compose([
    A.Normalize(0, 1),
    ToTensorV2(),
], additional_targets={'abc': 'image'})
ds = EuroSATDataset(trans=trans)
model = torchvision.models.resnet18()
model.fc = torch.nn.Linear(512, 10)

task = ImageClassification(model, inputs=['abc'], outputs=['def'])

trainer = pl.Trainer(
    gpus=1,
    precision=16,
    max_epochs=3,
    limit_train_batches=10
)

trainer.fit(task, ds)

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
extracting data ...: 100%|██████████| 27011/27011 [00:02<00:00, 11196.08it/s]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type             | Params
---------------------------------------------
0 | model   | ResNet           | 11.2 M
1 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
22.363    Total est

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


## Tasks are `LightningModule`s

Finally, note that `Tasks` are built on top of `Pytorch Lightning`'s `LightningModule`, so you can build your own tasks easily and use any of our `Datasets` (which are in turn `LightningDataModule`s).